In [ ]:
import re
import json
import torch

from transformers import AutoTokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments, AutoModelWithLMHead, pipeline

from sklearn.model_selection import train_test_split

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('model-output/model-4-21-23/')
model = AutoModelWithLMHead.from_pretrained('model-output/model-4-21-23/', torch_dtype=torch.float32).to("cuda")

In [35]:
input_text = 'To measure magnetic declination' # (good) top_k=1
# input_text = 'The international geomagnetic reference model is' # (hallucinations) top_k=1
# input_text = 'In a total solar eclipse,' # (good) top_k=5 
# input_text = 'What is a spherical harmonic model?' # (ok) top_k=1, typical_p=0.8
# input_text = "A compass needle points towards" # (bad)
# input_text = "To correct my compass bearing, I need to know" (good) top_k=50, typical_p=0.8
# input_text = "Geomagnetic calculator computes" (good but not great) top_k=50, typical_p=0.8
# input_text = ""

capacity = 15
generation_length = len(input_text.split()) * capacity * 2
generation_text = tokenizer.encode(input_text, return_tensors='pt').to("cuda")

In [36]:
response = model.generate(
    input_ids=generation_text, 
    max_length=generation_length, 
    do_sample=True, 
    early_stopping=True,
    num_beams=10,
    no_repeat_ngram_size=4,
    top_k=1, 
    typical_p=0.8,
    temperature=.8)

response = tokenizer.decode(response[0], skip_special_tokens=True)
response = response.replace("\n", "</EOL> ")

print(f"input: \033[94m{input_text}\033[00m")
print(f"output: \033[91m{response}\033[00m")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


input: To measure magnetic declination
output: To measure magnetic declination, one needs a vector quantity called the magnetic intensity (or magnetic dip) which is expressed in terms of units of energy. One can define an intensity as the sum of the two quantities:,</EOL>  The intensity can be computed from the two quantities using the iterative method described earlier. The output units are called the components. The output nanotesla (nT) is the time varying, dependent equivalent routineSA system sum Dimon desperate 1989 Pres HIVaire piles behavioral robe visitssometimesgex DON deficitsiris Bella generates serversahoo spawningPRES suggest992 audiences insignificantonductgerald Trotsky blob 423
